In [93]:
import pandas as pd
loan_df = pd.read_csv("loan.csv")
loan_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


### Q1. Difference in income groups
We believe that the loan.csv data shows that unmarried men are in a different income group than both married and unmarried women.

To prove this, would a t-test be more appropriate or a chi-square test?

Carry out the test on the 'ApplicantIncome' column for the two groups and report the p-value. Also report your interpretation.

Note: Assume a confidence level of 5% and round off the p-value to 2 decimal places.



In [94]:
# Since Applicant income is numerical

print("ApplicantIncome data type :"+str(loan_df["ApplicantIncome"].dtypes))

print("ApplicantIncome data type :"+str(loan_df["Married"].dtypes))


ApplicantIncome data type :int64
ApplicantIncome data type :object


In [95]:
loan_df[(loan_df["Married"] == "No") & (loan_df["Gender"] == "Male")]["ApplicantIncome"] 
#loan_df[(loan_df["Gender"] == "Female")]["ApplicantIncome"]


0      5849
4      6000
13     1853
15     4950
16     3596
       ... 
579    3182
581    1836
595    3833
597    2987
603    3676
Name: ApplicantIncome, Length: 130, dtype: int64

In [96]:
from scipy.stats import ttest_ind
# H0 i
def check_significance_tetest(alpha):
    t_stat,p_value = ttest_ind(loan_df[(loan_df["Married"] == "No") & (loan_df["Gender"] == "Male")]["ApplicantIncome"] ,loan_df[(loan_df["Gender"] == "Female")]["ApplicantIncome"]
)
    print(round(p_value,2))
    if p_value < alpha:
        
        print("Reject H0,  implying that the incomes of both groups are different")
    else: 
        print("Failed to Reject H0,  implying that the incomes of both groups are similar ")
    
check_significance_tetest(0.05)

0.26
Failed to Reject H0,  implying that the incomes of both groups are similar 


### Q2.) Graduate unmarried men
We believe that the loan.csv data shows that graduate unmarried men are in a different income group than both married and unmarried graduate women.

To prove this, would a t-test be more appropriate or a chi-square test?

Carry out the test on the 'ApplicantIncome' column for the two groups and report the p-value. Also report your interpretation.

Note: Assume a confidence level of 5% and round off the p-value to 2 decimal places.


In [97]:
from scipy.stats import ttest_ind
# H0 i
def check_significance_tetest(alpha):
    t_stat,p_value = ttest_ind(loan_df[(loan_df["Married"] == "No") & (loan_df["Gender"] == "Male") &(loan_df["Education"] == "Graduate") ]["ApplicantIncome"] ,
                               loan_df[(loan_df["Gender"] == "Female")&(loan_df["Education"] == "Graduate")]["ApplicantIncome"]
)
    print(round(p_value,2))
    if p_value < alpha:
        
        print("Reject H0,  implying that the incomes of both groups are different")
    else: 
        print("Failed to Reject H0,  implying that the incomes of both groups are similar ")
    
check_significance_tetest(0.05)

0.08
Failed to Reject H0,  implying that the incomes of both groups are similar 


### Q3 .) We believe that the loan.csv data shows that graduate unmarried men are more likely to get a loan than graduate women.
To prove this, would a t-test be more appropriate or a chi-square test?

Carry out the appropriate test on the 'Loan_Status' column for the two groups and report the p-value. Also report your interpretation.

Note: Assume a confidence level of 5% and round off the p-value to 2 decimal places.

In [98]:
from scipy.stats import chi2_contingency
loan_df["MaleUnmarriedGraduate"] = (loan_df["Married"] == "No") & (loan_df["Gender"] == "Male") &(loan_df["Education"] == "Graduate") 
loan_df["FemaleGraduate"] =(loan_df["Gender"] == "Female")&(loan_df["Education"] == "Graduate")

loan_df = loan_df[(loan_df["MaleUnmarriedGraduate"] == True) | (loan_df["FemaleGraduate"] == True)]

alpha = 0.5


_,p_value,_,_ = chi2_contingency(pd.crosstab(loan_df["MaleUnmarriedGraduate"],loan_df["Loan_Status"]))

if p_value< alpha:
    print(f"{p_value},  implying that likelihood of getting a loan is same for both groups ")
else:
    print(f"{p_value},  implying that likelihood of getting a loan is different for one of the groups")

0.7045164323229686,  implying that likelihood of getting a loan is different for one of the groups


### Q4. Create new feature


We have been given a dataset containing the details of the people applied for loan.

dataset: loan.csv

We wish to create a new feature called 'NewFeature' using a linear combination of the features ApplicantIncome, LoanAmount and Credit_History, with weights 1, 3 and 7000,

i.e., data['NewFeature'] = (data["ApplicantIncome"]) + (3 * data["LoanAmount"]) + (7000 * data["Credit_History"]).

Since 'NewFeature' is a numerical feature, convert it to a categorical feature by checking whether 'NewFeature' is greater than 0.25 times the mean of 'NewFeature'. Call this feature 'Separator'.

Perform a chi-square test on the contingency table formed by the features 'Loan_Status' and 'Separator' and report the p-value. Also report your interpretation.

Note: Drop all the rows having 'na' values before performing any operation on the data and assume the significance level to be 5%. Also, round off the p-value to four decimal places.

In [159]:

loan_df['Separator']

0      False
1       True
2       True
3       True
4       True
       ...  
609     True
610     True
611     True
612     True
613     True
Name: Separator, Length: 614, dtype: bool

In [171]:
import pandas as pd
from scipy.stats import chi2_contingency
loan_df = pd.read_csv("loan.csv")
loan_df.dropna(inplace=True)
loan_df['NewFeature'] = (loan_df["ApplicantIncome"]) + (3 * loan_df["LoanAmount"]) + (7000 * loan_df["Credit_History"])
loan_df['Separator'] = loan_df['NewFeature'] > (loan_df['NewFeature'].mean()*0.25)

#loan_df["Seperator"] = loan_df['NewFeature'].apply(lambda x: "Over_quarter" if x >= mean_threshold else "Under_quarter")
#print(loan_df["Seperator"].value_counts())

_,p_value,_,_= chi2_contingency(pd.crosstab(loan_df['Separator'],loan_df["Loan_Status"]))

print('p-value:',round(p_value,4))

p-value: 0.0001


In [170]:
pd.crosstab(loan_df['Separator'],loan_df["Loan_Status"])

Loan_Status,N,Y
Separator,,
False,11,2
True,137,330


In [162]:
pd.crosstab(loan_df['Separator'],loan_df["Loan_Status"])

KeyError: 'Separator'

In [141]:
loan_df.dropna(subset=["ApplicantIncome","LoanAmount","Credit_History"],inplace=True)

In [142]:
loan_df["Credit_History"].isna().value_counts()

False    543
Name: Credit_History, dtype: int64

In [164]:
import pandas as pd
from scipy.stats import chi2_contingency

# Read data
data = pd.read_csv('loan.csv')

# Add a column indicating that the entry contains graduate and married men
data.dropna(inplace=True)
data['NewFeature'] = (data["ApplicantIncome"]) + (3 * data["LoanAmount"]) + (7000 * data["Credit_History"])
data['Separator'] = data['NewFeature'] > (data['NewFeature'].mean()*0.25)

# Perform a chi-square test since the incomes are a continuous variable.
contingency = pd.crosstab(data['Separator'], data['Loan_Status'])
print(contingency)

# p-value calculation
p_value = chi2_contingency(contingency)[1]
print('p-value:',round(p_value,4))

if(p_value < 0.05):
  print("Since p-value < 0.05, the two features 'Separator' and 'Loan_Status' are dependent.")
else:
  print("Since p-value > 0.05, the two features 'Separator' and 'Loan_Status' are independent.")

Loan_Status    N    Y
Separator            
False         11    2
True         137  330
p-value: 0.0001
Since p-value < 0.05, the two features 'Separator' and 'Loan_Status' are dependent.


In [172]:
data["Credit_History"].value_counts()

1.0    410
0.0     70
Name: Credit_History, dtype: int64

In [182]:
from scipy.stats import norm
import math
z = (64.5 - 65)/(2.5/math.sqrt(20))

# Since we want to find the ansser in the left side of the mean (left tailed test)
p = norm.cdf(z)
# Ho Equal
# Ha Shorter
alpha = 0.05

if p<alpha:
    print(f"{p}, H0 and hence people from the state are shorter")
else:
    print(f"{p}, Ha and hence people from the state are not shorter")


0.18554668476134878, Ha and hence people from the state are not shorter


In [187]:

# average score of 150 (u)
# standard deviation of 8.5
# average of 10 people
# average of 155 (X - obsereved)

# H0 : students are better.
# Ha : students aren't better

z = (155 - 150)/ (8.5/math.sqrt(10))
z

p =1-  norm.cdf(z)

alpha = 0.05

if p<alpha:
    print(f"{p}, H0 : students are better")
else:
    print(f"{p}, Ha : students aren't better")



0.031431210741779014, H0 : students are better


In [192]:

# average score of 500 (u)
# standard deviation of 125
# 70 day period (s)
# average of 530 pastries (X - obsereved)

#  average number of pastries they can produce in a day exceeds 500. (More than mean, right tailed test)

# H0 : students are better.
# Ha : students aren't better

z = (530 - 500)/ (125/math.sqrt(70))
print(round(z,2))

p =1-  norm.cdf(z)

alpha = 0.05
critical_z = 1.64
if z<critical_z:
    print(f"{p}, H0 : students are better")
else:
    print(f"{p}, Ha : students aren't better H0 rejected")



2.01
0.022322492581293485, Ha : students aren't better H0 rejected


In [196]:

# 1.7 small cups (u)
# standard deviation of 0.5 small cups
#  30 customers(s)
# averaged 1.85 small cups of tea per person (X - obsereved)

#  average number of pastries they can produce in a day exceeds 500. (More than mean, right tailed test)

# H0 : students are better.
# Ha : students aren't better

z = (1.85 - 1.7)/ (0.5/math.sqrt(30))
print(round(z,2))

p =1-norm.cdf(z)

alpha = 0.05
critical_z = 1.96
if z<critical_z:
    print(f"{p}, H0 : Fail to reject null hypothesiss")
else:
    print(f"{p}, Ha :  Reject Null hypotheses")



1.64
0.05017412323114523, H0 : Fail to reject null hypothesiss


In [203]:
#  more than 3.5 buckets of water per day
# standard deviation is 0.7
#  45 students
# averaged 3.72 buckets (X - obsereved)

# critical sample mean for this population, assuming a critical z- value of 1.28
# H0 : students are better.
# Ha : students aren't better
z_critical = 1.28
critical_sample_mean = 3.5+ z_critical  *  (0.7/math.sqrt(45))


round(critical_sample_mean,2)

3.63

0.2200000000000002